# Nodeflux Face Enrollment (in-depth) DEMO

### Introduction

Welcome! In this demo, you will experience the nodeflux's Face Recognition products, especially on how to deal with face enrollments system with some provided options.

### Dependencies

Run below script once to install all required dependencies

In [ ]:
%pip install pandas
%pip install pillow
%pip install gradio
%pip install requests

### Face Enrollment - Configuration 

This section script will intialize all reusable functions and variable accross different Face Enrollment methods

In [36]:
# @title #### Fremis Config
FREMIS_ADDRESS = "192.168.103.46" # @param {type:"string"}
FREMIS_PORT = "2210" # @param {type:"string"}
FE_ENDPOINT = "v1/face/enrollment"

import requests
import json
from io import BytesIO
import base64

url = f"http://{FREMIS_ADDRESS}:{FREMIS_PORT}/{FE_ENDPOINT}"

# Convert Image to Base64 String Data
def image2base64(image):
    img_byte_array = BytesIO()
    image.save(img_byte_array, format='JPEG')
    img_byte_array = img_byte_array.getvalue()
    
    return base64.b64encode(img_byte_array).decode('utf-8')

# Enrollment Function
def enroll(image, face_id=None):
    headers = {
        'Content-Type': 'application/json'
    }

    body = {
        'image': image2base64(image),
        'keyspace': 'default',
    }

    if face_id:
        body['additional_params'] = {
            'face_id': str(face_id)
        }

    payload = json.dumps(body)
    response = requests.request("POST", url, headers=headers, data=payload)
    res = response.json()

    try:
        face_id = res['face_id']
        return {
            'face_id': res['face_id'],
            'success': True
        }
    except:
        return {
            'face_id': face_id,
            'success': False
        }

### Face Enrollment - Bulk Inserts

In [39]:

import time 
from PIL import Image

import gradio as gr
import pandas as pd

df = pd.read_csv('bulk_inputs/face_data.csv')

def bulk_enroll(in_df):
    file_names = []
    face_ids = []
    successes = []

    for _, row in in_df.iterrows():
        img = Image.open(row['file_name'])

        res = enroll(img)
        file_names.append(row['file_name'])
        face_ids.append(res['face_id'])
        successes.append(res['success'])

        time.sleep(0.1)

    out_df = pd.DataFrame(data={
            'file_name': file_names,
            'face_id': face_ids,
            'success': successes
        })
    
    return out_df

iface = gr.Interface(
    fn=bulk_enroll,  
    inputs=gr.Dataframe(datatype="str", row_count=(3, "dynamic"), col_count=1, interactive=False),
    outputs=gr.Dataframe(datatype="str", row_count=(3, "dynamic"), col_count=3, interactive=False), 
    title="Face Enrollment (Bulk with Custom Face ID)",
    examples=[
        df,
    ]
)

iface.launch()

Running on local URL:  http://127.0.0.1:7886

To create a public link, set `share=True` in `launch()`.


### Face Enrollment - Bulk Inserts with Custom Face IDs

In [40]:
import time 
from PIL import Image

import gradio as gr
import pandas as pd

url = f"http://{FREMIS_ADDRESS}:{FREMIS_PORT}/{FE_ENDPOINT}"

df = pd.read_csv('bulk_inputs/face_data_with_custom_id.csv')

def bulk_enroll_with_id(in_df):
    file_names = []
    face_ids = []
    successes = []

    for _, row in in_df.iterrows():
        img = Image.open(row['file_name'])

        res = enroll(img, row['face_id'])
        file_names.append(row['file_name'])
        face_ids.append(res['face_id'])
        successes.append(res['success'])

        time.sleep(0.1)

    out_df = pd.DataFrame(data={
            'file_name': file_names,
            'face_id': face_ids,
            'success': successes
        })
    
    return out_df

iface = gr.Interface(
    fn=bulk_enroll_with_id,  
    inputs=gr.Dataframe(datatype="str", row_count=(3, "dynamic"), col_count=2, interactive=False),
    outputs=gr.Dataframe(datatype="str", row_count=(3, "dynamic"), col_count=3, interactive=False), 
    title="Face Enrollment (Bulk with Custom Face ID)",
    examples=[
        df,
    ]
)

iface.launch()

Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


### Face Enrollment - Bulk Inserts with Custom Metadata via User Dashboard API

In [54]:
# @title #### Fremis Config
VANILLA_ADDRESS = "192.168.103.46" # @param {type:"string"}
VANILLA_PORT = "8008" # @param {type:"string"}
FE_ENDPOINT = "api/enrollment"

In [61]:
from io import BytesIO
import requests
from PIL import Image

url = f"http://{VANILLA_ADDRESS}:{VANILLA_PORT}/{FE_ENDPOINT}"

def image2bin(image):
    img_byte_array = BytesIO()
    image.save(img_byte_array, format='JPEG')
    return img_byte_array.getvalue()


def dashboard_enroll(image, payload):
    files = {
        ('images',('test.jpeg',image2bin(image),'image/jpeg'))
    }

    response = requests.request("POST", url, data=payload, files=files)
    res = response.json() 

    img_thumb = None
    try:
        base64_image_string = res['enrollment']['faces'][0]['image_thumbnail']
        decoded_bytes = base64.b64decode(base64_image_string)
        image_stream = BytesIO(decoded_bytes)
        img_thumb = Image.open(image_stream)

        return {
            'face_id': str(res['enrollment']['face_id']),
            'name': res['enrollment']['name'],
            'status': res['enrollment']['status'],
            'success': True,
        }, img_thumb
    except:
        return {
            'face_id': None,
            'name': None,
            'status': None,
            'success': True,
        }, None

In [ ]:
import time 
from PIL import Image

import gradio as gr
import pandas as pd

df = pd.read_csv('bulk_inputs/face_data_with_metadata.csv')

def bulk_enroll_with_metadata(in_df):
    file_names = []
    face_ids = []
    names = []
    status = []
    successes = []
    img_thumbs = []

    for _, row in in_df.iterrows():
        img = Image.open(row['file_name'])

        payload = {
            'name': row['name'],
            # 'face_id': row['face_id'] # Enable this if want to use Custom Face ID, instead randomly generated
            'identity_number': row['id_num'],
            'gender': row['gender'],
            'birth_date': row['birth_date'],
            'birth_place': row['birth_place'],
            'status': row['status'],
        }

        res, img_thumb = dashboard_enroll(img, payload)
        file_names.append(row['file_name'])
        face_ids.append(res['face_id'])
        names.append(res['name'])
        status.append(res['status'])
        successes.append(res['success'])
        if img_thumb:
            img_thumbs.append((img_thumb, res['face_id']))

        time.sleep(0.1)

    out_df = pd.DataFrame(data={
            'file_name': file_names,
            'face_id': face_ids,
            'name': names,
            'status': status,
            'success': successes
        })
    
    return out_df, img_thumbs

iface = gr.Interface(
    fn=bulk_enroll_with_metadata,  
    inputs=gr.Dataframe(datatype="str", row_count=(3, "dynamic"), col_count=7, interactive=False),
    outputs=[gr.Dataframe(datatype="str", row_count=(3, "dynamic"), col_count=5, interactive=False),
             gr.Gallery(label="Image Thumbnails", show_label=True, elem_id="gallery",
                        columns=[3], rows=[1], object_fit="contain", height="auto")], 
    title="Face Enrollment (Bulk with Custom Face ID)",
    examples=[
        df,
    ]
)

iface.launch()